<a href="https://colab.research.google.com/github/chopstickexe/deep-learning-from-scratch-2/blob/master/ch07_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 下準備

$$
\newcommand{\vect}[1]{\mathbf{#1}}
\newcommand{\mat}[1]{\mathbf{#1}}
$$

## 数式の表記

変数: 小文字イタリック $x$

定数: 大文字イタリック $X$

ベクトル: 小文字ローマン体太字 $\vect{x}$

行列: 大文字ローマン体太字 $\mat{X}$

## 公式実装のclone

In [1]:
!git clone --depth=1 https://github.com/oreilly-japan/deep-learning-from-scratch-2.git
import sys 
sys.path.append('deep-learning-from-scratch-2')

Cloning into 'deep-learning-from-scratch-2'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 73 (delta 13), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (73/73), done.


# 7章 RNNによる文章生成

6章で実装した言語モデルが出力する確率分布に従って単語をサンプリングし，文章を生成する．

## 7.1 言語モデルを使った文章生成（`RnnlmGen`，`BetterRnnlmGen`クラス）

In [2]:
import numpy as np
from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm

class RnnlmGen(Rnnlm):
  def generate(self, start_id, skip_ids=None, sample_size=100): 
    '''
    start_id: 開始単語のID
    skip_ids: <unk>など生成に使いたくない単語のリスト
    sample_size=文長
    '''
    word_ids = [start_id]

    x = start_id
    while len(word_ids) < sample_size:
      x = np.array(x).reshape(1, 1)
      score = self.predict(x)  # Softmaxレイヤの直前の出力
      p = softmax(score.flatten())  # flatten(): ndarrayを一次元の配列化する

      sampled = np.random.choice(len(p), size=1, p=p)  # サンプリング．len(p)は生成されるサンプルの値域最大値，sizeは生成されるサンプル数，pはサンプルする際に使われる確率分布

      if (skip_ids is None) or (sampled not in skip_ids):
        x = sampled
        word_ids.append(int(x))

    return word_ids

PTBデータセットで学習した普通のLSTMモデルを利用して文章生成．

In [3]:
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
model.load_params('deep-learning-from-scratch-2/ch06/Rnnlm.pkl')

# start文字とskip文字の設定
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 文章生成
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)

Done
you lose a step we said it would submit to post new sources ad.
 for the wives process day under the slowdown champion beneath control plans to be eliminated to work standards said auditors stemming from epo but is rushing to edge four products.
 the scramble now noting providing warsaw 's share of fears more worse demand and tucson credits and any premiums engaged to solve clients of quality or shipment guest.
 but in recession the poll consortium is an outside consumer groups are reasonable or on the issue gasb to ensure why those developers remain fairly


LSTMレイヤを2層にして，各層にDropoutを入れ，さらにEmbeddingの転置行列（H x V, Hは入力単語を表現する隠れベクトルのサイズ，Vは元の語彙数）をAffine Layerの重みにそのまま使うモデル（`ch06/better_rnnlm.py`）に変更すると，生成される文章の質が上がる．
（より言語のルールを守った文章になる）

In [4]:
!curl -o deep-learning-from-scratch-2/ch06/BetterRnnlm.pkl https://www.oreilly.co.jp/pub/9784873118369/BetterRnnlm.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 37.7M  100 37.7M    0     0  9222k      0  0:00:04  0:00:04 --:--:-- 9222k


In [6]:
from ch06.better_rnnlm import BetterRnnlm  # LSTMレイヤを2層利用して、各層でDropoutするモデル
from ch07.rnnlm_gen import BetterRnnlmGen  # RnnlmGenの継承クラスをRnnlmからBetterRnnlmに変更しただけのクラス

model = BetterRnnlmGen()
model.load_params('deep-learning-from-scratch-2/ch06/BetterRnnlm.pkl')

# start文字とskip文字の設定
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 文章生成
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)

you are a last-minute drain to the court.
 but the brief for extra systems and the national institutes of health after a national institutes of health prohibit a sometimes strong role in the centerpiece of congress and its account for data council in school.
 new spy rooms.
 lawyers for pregnant women the network salesmen have illegally for private scientific suggest that he would do a good job in the k mart fla. ad agency.
 the dispute between westinghouse and mr. smith said the price of in big personnel are more limited by both consumers and magazines


## 7.2 seq2seq（足し算問題）

足し算seq2seqを実装するためのデータセットを確認．
Decoder用の開始文字として`_`を入れている． 

In [7]:
from dataset import sequence

(x_train, t_train), (x_test, t_test) = \
  sequence.load_data('addition.txt', seed=1984)
char_to_id, id_to_char = sequence.get_vocab()

print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)

print(x_train[0])
print(t_train[0])

print(''.join([id_to_char[c] for c in x_train[0]]))
print(''.join([id_to_char[c] for c in t_train[0]]))

(45000, 7) (45000, 5)
(5000, 7) (5000, 5)
[ 3  0  2  0  0 11  5]
[ 6  0 11  7  5]
71+118 
_189 


## 7.3 seq2seqの実装

まずEncoderクラスを実装する．各時刻（各入力）はEmbeddingとLSTMの二層にする．

Encoderクラスの出力として欲しいのは一番最後の時刻が出す隠れベクトルだけなので（ここ重要．最終時刻のセルも要らない），他の時刻のLSTMの隠れベクトルやセルは特に使わない．

In [8]:
class Encoder:
  def __init__(self, vocab_size, wordvec_size, hidden_size):
    V, D, H = vocab_size, wordvec_size, hidden_size
    rn = np.random.randn

    embed_W = (rn(V, D) / 100).astype('f')
    lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
    lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
    lstm_b = np.zeros(4 * H).astype('f')

    self.embed = TimeEmbedding(embed_W)
    self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=False)
    
    self.params = self.embed.params + self.lstm.params
    self.grads = self.embed.grads + self.lstm.grads
    self.hs = None

  def forward(self, xs):
    xs = self.embed.forward(xs)
    hs = self.lstm.forward(xs)  # セルはそもそもforward()の戻り値として返って来ない仕様になっている
    self.hs = hs
    return hs[:, -1, :]  # 最終時刻のhのみ返す

  def backward(self, dh):
    dhs = np.zeros_like(self.hs)
    dhs[:, -1, :] = dh  # 最終時刻のhの勾配をセットする

    dout = self.lstm.backward(dhs)
    dout = self.embed.backward(dout)
    return dout

次にDecoderクラスを実装する．Encoderクラスから最終時刻の隠れベクトルを受け取り，RNNで文章を生成するが，今回は足し算の答えを出したいので，次の時刻の文字はSoftmaxに基づくサンプリングで決めるのではなく，一番高い確率値のものを決定的に出す．

そのため，シンプルなRNNとして実装したEmbedding, LSTM, Affine, SoftmaxWithLossのうち，最後のSoftmaxWithLossを省略して，AffineまでをDecoderとする．（SoftmaxWithLoss部分は後で実装するSeq2seqクラスで扱う．